In [1]:
import sys
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 3 -p
%matplotlib inline


## imports

In [7]:
from devtools import pprint

Reloading 'ocrorchestrator.utils.constants'.
Reloading 'ocrorchestrator.repos.factory'.


In [8]:
from ocrorchestrator.repos.factory import RepoFactory

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../secrets/calm-producer-428509-t9-b428a168489d.json"

Reloading 'ocrorchestrator.repos.local'.
Reloading 'ocrorchestrator.repos.factory'.


In [23]:
os.environ["REPO_TYPE"] = "gcs"
os.environ["REMOTE_PATH"] = "ocrorchestrator"

## testing repo

In [31]:
repo = RepoFactory.create_repo(os.environ["REPO_TYPE"])

Reloading 'ocrorchestrator.datamodels.api_io'.


In [36]:
from fastapi import HTTPException

Reloading 'ocrorchestrator'.
Reloading 'ocrorchestrator.repos.gcs'.
Reloading 'ocrorchestrator.repos'.
Reloading 'ocrorchestrator.utils.constants'.
Reloading 'ocrorchestrator.repos.local'.
Reloading 'ocrorchestrator.datamodels.api_io'.
Reloading 'ocrorchestrator.repos.base'.
Reloading 'ocrorchestrator.repos.factory'.


In [40]:
repo.download_obj("models/rs.pt")

Reloading 'ocrorchestrator.repos.gcs'.
{"status_code": 702, "status": "REPO_OBJECT_DOWNLOAD_ERROR", "event": "Repo operation error in download_obj", "level": "error", "exception": "Traceback (most recent call last):\n  File \"/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/cloud/storage/blob.py\", line 4353, in _prep_and_do_download\n    self._do_download(\n  File \"/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/cloud/storage/blob.py\", line 1023, in _do_download\n    response = download.consume(transport, timeout=timeout)\n  File \"/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/resumable_media/requests/download.py\", line 263, in consume\n    return _request_helpers.wait_and_retry(\n  File \"/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/resumable_media/requests/_request_helpers.py\", line 155, in wait_and_retry\n   

RepoException: 702: REPO_OBJECT_DOWNLOAD_ERROR: Traceback (most recent call last):
  File "/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/cloud/storage/blob.py", line 4353, in _prep_and_do_download
    self._do_download(
  File "/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/cloud/storage/blob.py", line 1023, in _do_download
    response = download.consume(transport, timeout=timeout)
  File "/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/resumable_media/requests/download.py", line 263, in consume
    return _request_helpers.wait_and_retry(
  File "/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/resumable_media/requests/_request_helpers.py", line 155, in wait_and_retry
    response = func()
  File "/home/ajkdrag/workspace/projects/ocrorchestrator/.venv/lib/python3.9/site-packages/google/resumable_media/requests/downloa